In [ ]:
#pip install selenium beautifulsoup4 pandas ollama

In [15]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import ollama

# ✅ ScraperAPI Key (Use Your Own https://www.scraperapi.com/)
API_KEY = "f9574aed95b57b6b216ce5baea15bb6a"

# ✅ IMDb Top 250 URL
imdb_url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"

# ✅ ScraperAPI URL to bypass bot detection
proxy_url = f"http://api.scraperapi.com?api_key={API_KEY}&url={imdb_url}&render=true"

# ✅ Fetch IMDb page through ScraperAPI
response = requests.get(proxy_url)

# ✅ Check if request was successful
if response.status_code == 200:
    html = response.text
    print("✅ IMDb page fetched successfully. HTML Length:", len(html))
else:
    print("❌ Failed to fetch IMDb page. Status Code:", response.status_code)
    exit()

# ✅ Parse IMDb HTML with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
movies = []

# ✅ Extracting movie titles & ratings
movie_rows = soup.select("li.ipc-metadata-list-summary-item")

for movie in movie_rows:
    title = movie.find("h3")
    rating = movie.find("span", class_="ipc-rating-star")
    
    if title and rating:
        movies.append({"Title": title.text.strip(), "Rating": rating.text.strip()})

# ✅ Convert extracted data to a Pandas DataFrame
df = pd.DataFrame(movies)

# ✅ Save extracted data to CSV file
df.to_csv("imdb_top_250.csv", index=False)

# ✅ Display top 10 results
print("\n🎬 Top 10 Movies Extracted from IMDb:\n")
print(df.head(10))

# ✅ Use Ollama (Qwen-2.5) for AI Summary (Optional)
if not df.empty:
    movie_list_text = "\n".join([f"{movie['Title']} - {movie['Rating']}" for movie in movies])

    prompt = f"Here is a list of top 250 IMDb movies with ratings:\n{movie_list_text}\nSummarize the best ones."
    response = ollama.chat(model="qwen2.5", messages=[{"role": "user", "content": prompt}])

    print("\n📌 Ollama AI Summary:\n")
    print(response['message']['content'])

✅ IMDb page fetched successfully. HTML Length: 2811338

🎬 Top 10 Movies Extracted from IMDb:

                                               Title      Rating
0                        1. The Shawshank Redemption    9.3 (3M)
1                                   2. The Godfather  9.2 (2.1M)
2                                 3. The Dark Knight    9.0 (3M)
3                           4. The Godfather Part II  9.0 (1.4M)
4                                    5. 12 Angry Men  9.0 (908K)
5   6. The Lord of the Rings: The Return of the King  9.0 (2.1M)
6                                7. Schindler's List  9.0 (1.5M)
7                                    8. Pulp Fiction  8.9 (2.3M)
8  9. The Lord of the Rings: The Fellowship of th...  8.9 (2.1M)
9                 10. The Good, the Bad and the Ugly  8.8 (842K)

📌 Ollama AI Summary:

Here's a summary of some of the top-rated movies in this list, focusing on those with high IMDB scores and critical acclaim:

1. **Demon Slayer: Kimetsu no Yaiba - Tsuz